#this notebook handles the python side of sampling positive and negative cases for model training.
###Results are of the form (A,At,B,Bt,reltype)
###Results are fed to DB2, which generates feature data for each sample.

In [1]:
import jaydebeapi
import jpype

In [2]:
%matplotlib inline
import pandas
import os
import numpy
from ordereddict import OrderedDict
from ggplot import *
import re

In [3]:
jar = '/gsa/yktgsa/home/v/m/vmisra/kg/lib/db2jcc4-10.5.0.4-2.95.0-SNAPSHOT.jar'
args='-Djava.class.path=%s' % jar
jvm = jpype.getDefaultJVMPath()
jpype.startJVM(jvm, args)

In [60]:
conn = jaydebeapi.connect('com.ibm.db2.jcc.DB2Driver',['jdbc:db2://kg.almaden.ibm.com:50000/KG_R229','db2inst1','perfload!'])
curs = conn.cursor()

In [64]:
conn = jaydebeapi.connect('com.ibm.db2.jcc.DB2Driver',['jdbc:db2://wd-app03.swg.usma.ibm.com:50000/KG_R231','db2inst1','Pa88w0rd'])
curs = conn.cursor()

In [65]:
#quick test of the connection
N_ENTRIES = 10
sql = 'select "subject/instanceName_up", "subject/typeType","object/instanceName_up","object/typeType"'
sql = sql + ' from ADS."SIRE1.1"'
sql = sql + ' fetch first %(N_ENTRIES)i rows only' % {'N_ENTRIES':N_ENTRIES}
print sql
curs.execute(sql)
out = curs.fetchall()
print out[:1000]

select "subject/instanceName_up", "subject/typeType","object/instanceName_up","object/typeType" from ADS."SIRE1.1" fetch first 10 rows only
[(u'ISLANDS', u'GEOLOGICALOBJ', u'SEA', u'GEOLOGICALOBJ'), (u'JUDGES', u'PEOPLE', u'APPOINTED', u'EVENT_PERSONNEL'), (u'JUDGES', u'PEOPLE', u'HIGH COURT', u'ORGANIZATION'), (u'JUDICIAL AND LEGAL SERVICES COMMISSION', u'ORGANIZATION', u'APPOINTED', u'EVENT_PERSONNEL'), (u'JUSTICES', u'PEOPLE', u'COURT OF APPEAL', u'ORGANIZATION'), (u'LEADERS', u'PEOPLE', u'ANGUILLA', u'GPE'), (u'LEADERS', u'PEOPLE', u'WORLD', u'GPE'), (u'LEEWARD ISLANDS', u'GEOLOGICALOBJ', u'LESSER ANTILLES', u'LOCATION'), (u'LEGISLATIVE BRANCH', u'ORGANIZATION', u'ANGUILLA', u'GPE'), (u'LEGISLATIVE BRANCH', u'ORGANIZATION', u'NATIONAL ANTHEM OF ANGUILLA', u'ORGANIZATION')]


##Load relations from DB2

In [206]:
#constants for loading the data
MIN_MENTIONS=2
DB_V = '235'
dump_path = '/home/db2inst1/vinith/dump_v%(db_v)s_minmentions%(min_mentions)s.del' %{"db_v":DB_V,"min_mentions":MIN_MENTIONS}
sql = '''
connect to KG_R%(db_v)s;
export to %(dump_path)s of del
SELECT * FROM
(Select 
    "subject/instanceName_up",
    "subject/typeType",
    "object/instanceName_up",
    "object/typeType",
    "relCanonicalName",
    SUM("mAB_doc") as "mAB",
    count(*) as "n_docs"
from (
     Select
        "subject/instanceName_up",
        "subject/typeType",
        "object/instanceName_up",
        "object/typeType",
        "relCanonicalName",
        "documentID",
        count(*) as "mAB_doc"
     from 
     (select * from ADS."SIRE1.1"
     fetch first %(row_limit)i rows only)
     group by ( "subject/instanceName_up",
                "subject/typeType",
                "object/instanceName_up",
                "object/typeType",
                "relCanonicalName",
                "documentID"
              )
     )
group by (  "subject/instanceName_up",
            "subject/typeType",
            "object/instanceName_up",
            "object/typeType",
            "relCanonicalName"
         )
)
Where "mAB" > %(min_mab)i;''' % {"db_v":DB_V,"dump_path": dump_path, "min_mab": MIN_MENTIONS-1, "row_limit": 300000000}

print sql


connect to KG_R235;
export to /home/db2inst1/vinith/dump_v235_minmentions2.del of del
SELECT * FROM
(Select 
    "subject/instanceName_up",
    "subject/typeType",
    "object/instanceName_up",
    "object/typeType",
    "relCanonicalName",
    SUM("mAB_doc") as "mAB",
    count(*) as "n_docs"
from (
     Select
        "subject/instanceName_up",
        "subject/typeType",
        "object/instanceName_up",
        "object/typeType",
        "relCanonicalName",
        "documentID",
        count(*) as "mAB_doc"
     from 
     (select * from ADS."SIRE1.1"
     fetch first 300000000 rows only)
     group by ( "subject/instanceName_up",
                "subject/typeType",
                "object/instanceName_up",
                "object/typeType",
                "relCanonicalName",
                "documentID"
              )
     )
group by (  "subject/instanceName_up",
            "subject/typeType",
            "object/instanceName_up",
            "object/typeType",
            "r

###Run the above SQL in db2 on wd-app03.
###Then, copy over the file to the path: local_path_reltable

##Load Entity data from DB2

In [170]:
#creating vintitystats (custom version of ads.entitystats2)
sql = '''
drop table vintitystats;
create table vintitystats
    ("subject/instanceName_up" varchar(128), "subject/typeType" varchar(128), "RELATIONSHIPMENTIONCOUNT" varchar(128), "DOCUMENTCOUNT" varchar(128));

insert into vintitystats
    ("subject/instanceName_up", "subject/typeType", "RELATIONSHIPMENTIONCOUNT", "DOCUMENTCOUNT")
Select * from (
    Select "subject/instanceName_up", "subject/typeType", SUM("mA") as "RELATIONSHIPMENTIONCOUNT", COUNT("documentID") as "DOCUMENTCOUNT"
    from (
        SELECT "subject/instanceName_up", "subject/typeType", COUNT(*) as "mA", "documentID"
        from ADS."SIRE1.1"
        group by "subject/instanceName_up", "subject/typeType", "documentID"
        )
    group by "subject/instanceName_up", "subject/typeType"
    ); '''
print sql


drop table vintitystats;
create table vintitystats
    ("subject/instanceName_up" varchar(128), "subject/typeType" varchar(128), "RELATIONSHIPMENTIONCOUNT" varchar(128), "DOCUMENTCOUNT" varchar(128));

insert into vintitystats
    ("subject/instanceName_up", "subject/typeType", "RELATIONSHIPMENTIONCOUNT", "DOCUMENTCOUNT")
Select * from (
    Select "subject/instanceName_up", "subject/typeType", SUM("mA") as "RELATIONSHIPMENTIONCOUNT", COUNT("documentID") as "DOCUMENTCOUNT"
    from (
        SELECT "subject/instanceName_up", "subject/typeType", COUNT(*) as "mA", "documentID"
        from ADS."SIRE1.1"
        group by "subject/instanceName_up", "subject/typeType", "documentID"
        )
    group by "subject/instanceName_up", "subject/typeType"
    ); 


In [208]:
MIN_RELCOUNT=3
MAX_RELCOUNT=1000

sql = """
export to /home/db2inst1/vinith/dump_entities_v%(db_v)s_minrelcount_%(min_relcount)i.del of del
Select * from vintitystats
where ("DOCUMENTCOUNT" > %(min_docs)i) and
("DOCUMENTCOUNT" < %(max_docs)i)
fetch first %(row_limit)i rows only;
""" % {"db_v":DB_V, "min_relcount":MIN_RELCOUNT, "row_limit":100000000, "min_docs":MIN_RELCOUNT, "max_docs":MAX_RELCOUNT}
print sql


export to /home/db2inst1/vinith/dump_entities_v235_minrelcount_3.del of del
Select * from vintitystats
where ("DOCUMENTCOUNT" > 3) and
("DOCUMENTCOUNT" < 1000)
fetch first 100000000 rows only;



###Run the above SQL on wd-app03
###Copy over file to local_path_entitytable

##load both tables into pandas

In [121]:
relcols = ['A',
           'At',
           'B',
           'Bt',
           'reltype',
           'mAB',
           'nDocs']
local_path_reltable = '/ceph/vinith/kg/dump_rel.del'
reldata = pandas.read_csv(local_path_reltable,header=None,names=relcols)

In [122]:
reldata['pattern_A']=reldata.loc[:,'A']+';'+reldata.loc[:,'At']
reldata['pattern_B']=reldata.loc[:,'B']+';'+reldata.loc[:,'Bt']
reldata['pattern']=reldata.loc[:,'pattern_A']+';'+reldata.loc[:,'pattern_B']

In [119]:
#for MQT dump, which we are not currently using. Using manually generated table instead.
#entitycols = ['A','At','SomeCrap','nDocs','relcount','neighbors','relcount_distinct']
entitycols = ['A',
              'At',
              'relcount',
             'nDocs']
local_path_entitytable = '/ceph/vinith/kg/dump_entities.del'
entitydata = pandas.read_csv(local_path_entitytable,header=None,names=entitycols)
#entitydata = entitydata.drop('SomeCrap',1) #removing unnecessary info from MQT dump

##Positive Sampling

Idea: randomly sample significant relationships between significant entities

In [124]:
reldata=pandas.merge(reldata,
                                 entitydata,
                                 left_on=['A','At'],
                                 right_on=['A','At'],
                                 suffixes=['','_A'])
reldata

,A,At,B,Bt,reltype,mAB,nDocs,pattern_A,pattern_B,pattern,relcount,nDocs_A
0,""" AMARANTE",PERSON,PORTUGAL,GPE,locatedAt,4,4,""" AMARANTE;PERSON",PORTUGAL;GPE,""" AMARANTE;PERSON;PORTUGAL;GPE",4,4
1,""" ASIMOV",PERSON,FICTION,TITLEWORK,authorOf,6,4,""" ASIMOV;PERSON",FICTION;TITLEWORK,""" ASIMOV;PERSON;FICTION;TITLEWORK",18,6
2,""" ASTANA",PERSON,KAZAKHSTAN,GPE,locatedAt,6,6,""" ASTANA;PERSON",KAZAKHSTAN;GPE,""" ASTANA;PERSON;KAZAKHSTAN;GPE",10,8
3,""" BANGKOK",PERSON,THAILAND,GPE,locatedAt,6,6,""" BANGKOK;PERSON",THAILAND;GPE,""" BANGKOK;PERSON;THAILAND;GPE",11,6
4,""" BANGKOK",PERSON,SINGLES,EVENT_SPORTS,participantIn,5,5,""" BANGKOK;PERSON",SINGLES;EVENT_SPORTS,""" BANGKOK;PERSON;SINGLES;EVENT_SPORTS",11,6
5,""" BEL",PERSON,VL,ORGANIZATION,employedBy,12,12,""" BEL;PERSON",VL;ORGANIZATION,""" BEL;PERSON;VL;ORGANIZATION",35,30
6,""" BEL",PERSON,WA,ORGANIZATION,employedBy,18,18,""" BEL;PERSON",WA;ORGANIZATION,""" BEL;PERSON;WA;ORGANIZATION",35,30
7,""" BOTTOM 3",PERSON,COUPLES,PEOPLE,partOfMany,6,6,""" BOTTOM 3;PERSON",COUPLES;PEOPLE,""" BOTTOM 3;PERSON;COUPLES;PEOPLE",6,6
8,""" BUKHARA",PERSON,UZBEKISTAN,GPE,locatedAt,4,4,""" BUKHARA;PERSON",UZBEKISTAN;GPE,""" BUKHARA;PERSON;UZBEKISTAN;GPE",4,4
9,""" ELIM",PERSON,"#FBF373""",PERSON,colocation,4,4,""" ELIM;PERSON","#FBF373"";PERSON",""" ELIM;PERSON;#FBF373"";PERSON",15,7


In [125]:
reldata=pandas.merge(reldata,
                                 entitydata,
                                 left_on=['B','Bt'],
                                 right_on=['A','At'],
                                 suffixes=['','_B'])
reldata

,A,At,B,Bt,reltype,mAB,nDocs,pattern_A,pattern_B,pattern,relcount,nDocs_A,A_B,At_B,relcount_B,nDocs_B
0,""" ASIMOV",PERSON,FICTION,TITLEWORK,authorOf,6,4,""" ASIMOV;PERSON",FICTION;TITLEWORK,""" ASIMOV;PERSON;FICTION;TITLEWORK",18,6,FICTION,TITLEWORK,238,83
1,ASIMOV,PERSON,FICTION,TITLEWORK,authorOf,6,6,ASIMOV;PERSON,FICTION;TITLEWORK,ASIMOV;PERSON;FICTION;TITLEWORK,1362,352,FICTION,TITLEWORK,238,83
2,""" BEL",PERSON,VL,ORGANIZATION,employedBy,12,12,""" BEL;PERSON",VL;ORGANIZATION,""" BEL;PERSON;VL;ORGANIZATION",35,30,VL,ORGANIZATION,8,6
3,""" BEL",PERSON,WA,ORGANIZATION,employedBy,18,18,""" BEL;PERSON",WA;ORGANIZATION,""" BEL;PERSON;WA;ORGANIZATION",35,30,WA,ORGANIZATION,86,66
4,FREMANTLE,ORGANIZATION,WA,ORGANIZATION,colocation,5,5,FREMANTLE;ORGANIZATION,WA;ORGANIZATION,FREMANTLE;ORGANIZATION;WA;ORGANIZATION,717,314,WA,ORGANIZATION,86,66
5,FT. LEWIS,ORGANIZATION,WA,ORGANIZATION,colocation,4,4,FT. LEWIS;ORGANIZATION,WA;ORGANIZATION,FT. LEWIS;ORGANIZATION;WA;ORGANIZATION,15,7,WA,ORGANIZATION,86,66
6,JOINT BASE LEWIS-MCCHORD,ORGANIZATION,WA,ORGANIZATION,colocation,7,7,JOINT BASE LEWIS-MCCHORD;ORGANIZATION,WA;ORGANIZATION,JOINT BASE LEWIS-MCCHORD;ORGANIZATION;WA;ORGAN...,231,96,WA,ORGANIZATION,86,66
7,NAS WHIDBEY ISLAND,ORGANIZATION,WA,ORGANIZATION,colocation,6,6,NAS WHIDBEY ISLAND;ORGANIZATION,WA;ORGANIZATION,NAS WHIDBEY ISLAND;ORGANIZATION;WA;ORGANIZATION,169,60,WA,ORGANIZATION,86,66
8,NSW,ORGANIZATION,WA,ORGANIZATION,colocation,9,9,NSW;ORGANIZATION,WA;ORGANIZATION,NSW;ORGANIZATION;WA;ORGANIZATION,2003,935,WA,ORGANIZATION,86,66
9,QLD,GPE,WA,ORGANIZATION,colocation,5,5,QLD;GPE,WA;ORGANIZATION,QLD;GPE;WA;ORGANIZATION,638,380,WA,ORGANIZATION,86,66


In [176]:
N_SAMPLES = 2000

In [177]:
#sampling
rows = numpy.random.choice(reldata.index.values,N_SAMPLES)
samples_pos = reldata.ix[rows,['A','At','B','Bt']]
samples_pos

,A,At,B,Bt
777994,BLAISE PASCAL,PERSON,CLERMONT-FERRAND,GPE
1232571,NAVY COMMENDATION MEDAL,ORGANIZATION,COMBAT V,ORGANIZATION
387660,BOLSHEVIK PARTY,ORGANIZATION,PETROGRAD,GPE
602435,JUAN PERON,PERSON,REVOLUCION LIBERTADORA,ORGANIZATION
356339,NASJONAL SAMLING,PERSON,VIDKUN QUISLING,PERSON
1075254,ROBERT BENFIELD,PERSON,WILLIAM ECCLESTONE,PERSON
754091,BIG CYPRESS NATIONAL PRESERVE,ORGANIZATION,EVERGLADES NATIONAL PARK,ORGANIZATION
491337,CASEY JONES,PERSON,DONATELLO,PERSON
632921,AUGUSTE DUPIN,PERSON,THE MURDERS,TITLEWORK
512315,LAURA HILLENBRAND,PERSON,LOUIS ZAMPERINI,PERSON


##Negative Sampling

Idea: pick random rows from reldata_filtered, then use them as one of the following: AX, XA. 

Find an XB or BX in reldata_filtered for this choice.

Report back AB.

In [178]:
N_SAMPLES = 2000

In [179]:
rows = numpy.random.choice(reldata.index.values,N_SAMPLES)
negative_AX = reldata.ix[rows,:].reset_index()

In [180]:
count = 0
#helper function for sampling an AB using a given row as a starting point
def sample_AB(row):
    #print row
    #pdb.set_trace()
    global count
    count = count+1
    matches = ['AA','AB','BA','BB'] #indicates whether we're matching the A with the A, the A with the B, etc.
    relevant_Cs = OrderedDict()
    #find the rows in reldata_significant that match the row with any one of the 4 patterns.
    for match in matches:
        idxs = (reldata['pattern_'+match[0]]== row['pattern_'+match[1]])&~(reldata['pattern']==row['pattern'])
        relevant_Cs[match] = reldata[idxs]
        relevant_Cs[match]['match'] = match
    relevant_C = pandas.concat(relevant_Cs.values(),ignore_index=True)
    if len(relevant_C) == 0:
        #print 'null'
        return 'null'
    else:
        location = numpy.random.choice(relevant_C.index.values)
        match = relevant_C.ix[location,'match']
        opposite_match = matches[-1-matches.index(match)]
        pattern_A = row['pattern_'+opposite_match[1]]
        pattern_B = relevant_C.ix[location,'pattern_'+opposite_match[0]]
    print count, " ", pattern_A+';'+pattern_B
    return pattern_A+';'+pattern_B

def test(row):
    global count
    count = count +1
    print row
    pdb.set_trace()
    return '1'

In [156]:
negative_AXB = pandas.DataFrame()
negative_AXB['pattern_AB'] = negative_AX.apply(func=sample_AB, axis=1)
negative_AXB = negative_AXB[~(negative_AXB['pattern_AB']=='null')]


1   HUSH PUPPY;PERSON;SHARI LEWIS;PERSON
2   JOAN MURPHY;PERSON;KATE KELLY;PERSON
3   STANLEY KUBRICK;PERSON;AVRIL LAVIGNE;PERSON
4   JOEY LAWRENCE;PERSON;TOON DISNEY;PERSON
5   MIHAJLO KRESIMIR II;PERSON;SWEYN ESTRIDSON;PERSON


/usr/local/lib/python2.7/dist-packages/ipython-3.1.0-py2.7.egg/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [185]:
def split_AB(row,i=0):
    pieces = str(row['pattern_AB']).split(';')
    if (len(pieces)==4):
        return pieces[i]
    else:
        return 'null'

In [186]:
negative_AXB['A'] = negative_AXB.apply(lambda x: split_AB(x,0),axis=1)
negative_AXB['At'] = negative_AXB.apply(lambda x: split_AB(x,1),axis=1)
negative_AXB['B'] = negative_AXB.apply(lambda x: split_AB(x,2),axis=1)
negative_AXB['Bt'] = negative_AXB.apply(lambda x: split_AB(x,3),axis=1)
negative_AXB[['A','At','B','Bt']]

,A,At,B,Bt
0,EDMONTON OIL KINGS,PERSON,WINNIPEG JETS,ORGANIZATION
1,FIA WORLD ENDURANCE CHAMPIONSHIP,ORGANIZATION,FIA,ORGANIZATION
2,ROYAL SOCIETY OF PAINTERS,ORGANIZATION,PAINTERS,GPE
3,CA WOODWARD,GPE,PLATFORMS,ORGANIZATION
4,GRAJEWO COUNTY,GPE,GMINA AUGUSTOW,PERSON
5,PICKUP TRUCKS,VEHICLE,PEUGEOT,VEHICLE
6,GEORGE BENSON,PERSON,GOT TO GET YOU INTO MY LIFE,TITLEWORK
7,PRINCESS LOUISE,PERSON,ERNST FREDERICK II,PERSON
9,ALFRED L. BULWINKLE,PERSON,GEORGE H. MAHON,PERSON
10,PARRAMATTA STADIUM,PERSON,NORTH QUEENSLAND,FACILITY


In [187]:
set_negative_AXB = set([tuple(x) for x in negative_AXB[['A','At','B','Bt']].values.tolist()])
set_rel = set([tuple(x) for x in reldata[['A','At','B','Bt']].values.tolist()])
set_rel_reverse = set([tuple(x) for x in reldata[['B','Bt','A','At']].values.tolist()])
set_diff = set_negative_AXB - set_rel - set_rel_reverse

In [188]:
samples_neg = pandas.DataFrame(list(set_diff),columns=['A','At','B','Bt'])
samples_neg

,A,At,B,Bt
0,RAUNDALEN VALLEY,GPE,OSTEROY MUNICIPALITY,GPE
1,DANIEL DAE KIM,PERSON,DANNY WILLIAMS,PERSON
2,GOERLITZ,GPE,RABBI MEIR,PERSON
3,JACK WORRALL,PERSON,ROUND 7,PERSON
4,25PX PENNSYLVANIA ROUTE,PERSON,25PX WEST VIRGINIA ROUTE,PERSON
5,MAHESH MANJREKAR,PERSON,JUHI CHAWLA,PERSON
6,CHARLENE,PERSON,HOSNI MUBARAK,PERSON
7,GRAND ICE CREAM,FACILITY,EDWARD L.,PERSON
8,ASSOCIATION OF AMERICAN COLLEGES AND UNIVERSITIES,ORGANIZATION,AMERICAN ASSOCIATION OF STATE COLLEGES AND UNI...,ORGANIZATION
9,KING GEORGE STAKES,PERSON,CORONATION STAKES,PERSON


##Dump data to CSV, and then copy over to db2 machine for featurization

In [189]:
print len(samples_pos)
print len(samples_neg)

2000
1647


In [190]:
samples_pos['label']=numpy.ones(len(samples_pos))
samples_neg['label']=numpy.ones(len(samples_neg))*(-1)
samples = pandas.concat([samples_pos,samples_neg],ignore_index=True)
#samples

In [191]:
python_samples_name = 'datarank_samples_frompython.csv'
python_samples_localpath = '/ceph/vinith/kg/'+python_samples_name
samples.to_csv(python_samples_localpath,header=False,index=False)

###SCP this over to the DB2 machine

In [192]:
python_samples_db2machinepath = '/home/db2inst1/vinith/'+python_samples_name
argdict = {'localpath': python_samples_localpath,'output_path':python_samples_db2machinepath}
print "scp %(localpath)s db2inst1@wd-app03.swg.usma.ibm.com:%(output_path)s" % argdict

scp /ceph/vinith/kg/datarank_samples_frompython.csv db2inst1@wd-app03.swg.usma.ibm.com:/home/db2inst1/vinith/datarank_samples_frompython.csv


## Generate sql for featurization

In [197]:
sql_skeleton = open('/ceph/vinith/kg/featurizing_sql/featurize_skeleton.sql','r').read()
featurized_samples_db2machinepath ='/home/db2inst1/vinith/featurized_samples_estats2.del'
print sql_skeleton % {'input_path': python_samples_db2machinepath, 'output_path': featurized_samples_db2machinepath}


--create samples table from samples from python
drop table linkpred_samples;
create table linkpred_samples
	("s" varchar(128), "st" varchar(128), "o" varchar(128), "ot" varchar(128), "label" int);
import from /home/db2inst1/vinith/datarank_samples_frompython.csv of del insert into linkpred_samples("s","st","o","ot","label");

--featurize the samples and dump them to disk.
export to /home/db2inst1/vinith/featurized_samples_estats2.del of del
SELECT 
	A_X_B_estatsXA."label" as "label",
	A_X_B_estatsXA."A" as "A", 
	A_X_B_estatsXA."X" as "X",
	A_X_B_estatsXA."B" as "B", 
	A_X_B_estatsXA."At" as "At",
	A_X_B_estatsXA."Xt" as "Xt",
	A_X_B_estatsXA."Bt" as "Bt",
	A_X_B_estatsXA."AX" as "AX",
	A_X_B_estatsXA."XB" as "XB",
	A_X_B_estatsXA."directionXB",
	A_X_B_estatsXA."directionAX",
	A_X_B_estatsXA."mAX" as "mAX",
	A_X_B_estatsXA."mXB" as "mXB",
	A_X_B_estatsXA."docsAX" as "docsAX",
	A_X_B_estatsXA."docsXB" as "docsXB",
	A_X_B_estatsXA."entity_connections_X"  as "entity_connections_X",
	A_X_B

##SCP results of featurization sql back over to local machine

In [198]:
featurized_samples_localpath = '/ceph/vinith/kg/featurized_samples_estats2.del'
argdict = {'db2path': featurized_samples_db2machinepath, 'localpath': featurized_samples_localpath}
print "scp db2inst1@wd-app03.swg.usma.ibm.com:%(db2path)s %(localpath)s" %argdict

scp db2inst1@wd-app03.swg.usma.ibm.com:/home/db2inst1/vinith/featurized_samples_estats2.del /ceph/vinith/kg/featurized_samples_estats2.del
